In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import geopandas as gpd
import numpy as np
from geopy.distance import geodesic

temperaturas

In [20]:
# url de Historical Weather API con las coordenadas de hmo y los rangos de fecha buscados
url = 'https://archive-api.open-meteo.com/v1/archive?latitude=29.1026&longitude=-110.9773&start_date=2021-09-01&end_date=2024-11-13&daily=temperature_2m_mean&timezone=America%2FDenver&format=csv'

ruta_descarga = os.path.dirname(os.getcwd()) + '/data/raw/temp.csv'

# Realizar la descarga y guardar el archivo
response = requests.get(url)
with open(ruta_descarga, 'wb') as f:
    f.write(response.content)

# Cargar el archivo CSV local
df_temp_prueba = pd.read_csv(ruta_descarga)

df_temp_prueba

,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,29.06854,-110.975876,214.0,-25200.0,America/Denver,MST
1,time,temperature_2m_mean (°C),NaN,NaN,NaN,NaN
2,2021-09-01,26.4,NaN,NaN,NaN,NaN
3,2021-09-02,27.8,NaN,NaN,NaN,NaN
4,2021-09-03,27.8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1167,2024-11-09,19.3,NaN,NaN,NaN,NaN
1168,2024-11-10,19.9,NaN,NaN,NaN,NaN
1169,2024-11-11,20.2,NaN,NaN,NaN,NaN
1170,2024-11-12,NaN,NaN,NaN,NaN,NaN


In [21]:
coordenadas_zonas = [(29.109901548511832, -111.02333639953876), (29.124541070577358, -110.99291007309039), (29.14611675223518, -110.97739536308556), 
                     (29.082268147228778, -111.01454901072867), (29.090474375931272, -110.97106699419052), (29.097431318730578, -110.93146365549403),
                     (29.049870596836804, -110.98200537865017), (29.05662334236239, -110.95505454285305), (29.03321193425804, -110.94132163335122)
                     ]

# Crear un diccionario para almacenar las precipitaciones
data = {}

for i, coordenada in enumerate(coordenadas_zonas, start=1):
    url = f'https://archive-api.open-meteo.com/v1/archive?latitude={coordenada[0]}&longitude={coordenada[1]}&start_date=2021-09-01&end_date=2024-11-13&daily=temperature_2m_mean&timezone=America%2FDenver&format=csv'

    # Descargar el archivo CSV
    response = requests.get(url)
    # Nombramos cada columna por zona
    col_name = f'zona_{i}'
    
    # Guardar el archivo temporalmente
    ruta_descarga = os.path.dirname(os.getcwd()) + f'/data/raw/temp_zona_{i}.csv'
    with open(ruta_descarga, 'wb') as f:
        f.write(response.content)

    # Cargar el archivo CSV
    df_temp_hmo = pd.read_csv(ruta_descarga)

    # Quitamos las primeras dos filas inecesarias y reseteamos el indice
    df_temp_hmo.drop([0, 1], inplace=True)
    df_temp_hmo.reset_index(drop=True, inplace=True)

    # Quitamos columnas mal hechas debido a la estructura
    df_temp_hmo.drop(['elevation', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'], axis=1, inplace=True)
    # Renombramos adecuadamente las columnas que nos interesan
    df_temp_hmo.rename(columns={'latitude': 'fecha', 'longitude': 'temp'}, inplace=True)
    # Quitamos filas con nulos
    df_temp_hmo.dropna(inplace=True)

    # Definimos correctamente los tipos de datos
    df_temp_hmo['fecha'] = pd.to_datetime(df_temp_hmo['fecha'])
    df_temp_hmo['temp'] = df_temp_hmo['temp'].astype(float)
    
    # Asignamos la fecha como indice
    df_temp_hmo.set_index('fecha', inplace=True)
    
    # Añadir la columna de precipitación de esta zona al diccionario
    data[col_name] = df_temp_hmo['temp']

# Crear un DataFrame donde cada columna es una zona y los índices son las fechas
df_temps = pd.DataFrame(data)

In [22]:
df_temps

,zona_1,zona_2,zona_3,zona_4,zona_5,zona_6,zona_7,zona_8,zona_9
fecha,,,,,,,,,
2021-09-01,26.3,26.2,26.0,26.6,26.5,26.2,26.5,26.4,26.3
2021-09-02,27.7,27.6,27.3,28.0,27.9,27.7,27.9,27.8,27.7
2021-09-03,27.7,27.6,27.3,28.0,27.9,27.6,27.9,27.8,27.6
2021-09-04,28.2,28.1,27.8,28.3,28.2,28.0,28.2,28.2,27.9
2021-09-05,28.0,27.9,27.6,28.2,28.1,27.9,28.1,28.0,27.7
...,...,...,...,...,...,...,...,...,...
2024-11-07,19.2,19.1,18.8,19.2,19.1,18.9,19.1,19.0,18.7
2024-11-08,18.9,18.8,18.5,18.7,18.6,18.4,18.6,18.6,18.1
2024-11-09,19.7,19.6,19.4,19.5,19.4,19.1,19.4,19.3,19.0


baches y zonas

In [23]:
directorio = os.path.dirname(os.getcwd())

In [24]:
baches = gpd.read_file(directorio + '/data/processed/tidy_baches_agebs_hmo.shp')
baches.CVEGEO = baches.CVEGEO.str[-4:]
baches.head()

,latitude,longitude,CVEGEO,date,neighborho,descriptio,geometry
0,29.094161,-110.969471,1996,2021-10-18,[551],"Hay un bache en la calle, y los carros se golp...",POINT (-110.96947 29.09416)
1,29.124060,-110.971318,1746,2021-10-19,[601],"Buen dia! Para reportar pavimento dañado, bach...",POINT (-110.97132 29.12406)
2,29.123985,-110.971286,1746,2021-10-19,[601],"Buen dia! Para reportar pavimento dañado, bach...",POINT (-110.97129 29.12399)
3,29.128777,-110.953926,5043,2021-10-21,[49],toda la calle desde el blvd Morelos tiene vari...,POINT (-110.95393 29.12878)
4,29.083809,-111.042101,2617,2021-10-04,[74],C. SOLICITA BACHEO ASEGURA REPORTANTE QUE ES P...,POINT (-111.0421 29.08381)


In [25]:
df_baches = baches[['date', 'latitude', 'longitude']]
df_baches.rename(columns={'date': 'fecha'}, inplace=True)

# Lista de coordenadas de las zonas, de zona 1 a 9
coordenadas_zonas = [(29.109901548511832, -111.02333639953876), (29.124541070577358, -110.99291007309039), (29.14611675223518, -110.97739536308556), 
                     (29.082268147228778, -111.01454901072867), (29.090474375931272, -110.97106699419052), (29.097431318730578, -110.93146365549403),
                     (29.049870596836804, -110.98200537865017), (29.05662334236239, -110.95505454285305), (29.03321193425804, -110.94132163335122)
                     ]

# Función para encontrar la zona más cercana
def asignar_zona(lat_bache, lon_bache, zonas):
    distancias = [geodesic((lat_bache, lon_bache), (lat_zona, lon_zona)).km for lat_zona, lon_zona in zonas]
    zona_cercana = distancias.index(min(distancias))  # Encuentra la zona más cercana
    return zona_cercana + 1  # Usamos 1-based index (1 a 9) para las zonas

# Aplicar la función a cada reporte de bache
df_baches['zona'] = df_baches.apply(lambda row: asignar_zona(row['latitude'], row['longitude'], coordenadas_zonas), axis=1)

# Mostrar el DataFrame con la zona asignada a cada reporte de bache
print(df_baches)

C:\Users\sbrxb\AppData\Local\Temp\ipykernel_7020\4112246705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baches.rename(columns={'date': 'fecha'}, inplace=True)


           fecha   latitude   longitude  zona
0     2021-10-18  29.094161 -110.969471     5
1     2021-10-19  29.124060 -110.971318     2
2     2021-10-19  29.123985 -110.971286     2
3     2021-10-21  29.128777 -110.953926     3
4     2021-10-04  29.083809 -111.042101     4
...          ...        ...         ...   ...
18828 2024-11-11  29.165260 -111.001808     3
18829 2024-11-11  29.164831 -111.001746     3
18830 2024-11-11  29.065486 -110.989319     7
18831 2024-11-11  29.105208 -110.981033     5
18832 2024-11-11  29.105140 -110.962317     5

[18833 rows x 4 columns]


C:\Users\sbrxb\AppData\Local\Temp\ipykernel_7020\4112246705.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baches['zona'] = df_baches.apply(lambda row: asignar_zona(row['latitude'], row['longitude'], coordenadas_zonas), axis=1)


merge de baches y temperaturas por zonas

In [26]:
# Unir los dos DataFrames por fecha
df_baches_temps = pd.merge(df_baches, df_temps, how='left', on='fecha')

df_baches_temps.dropna(inplace=True)

df_baches_temps.set_index('fecha', inplace=True)

df_baches_temps

,latitude,longitude,zona,zona_1,zona_2,zona_3,zona_4,zona_5,zona_6,zona_7,zona_8,zona_9
fecha,,,,,,,,,,,,
2021-10-18,29.094161,-110.969471,5,24.1,24.0,23.8,24.1,24.0,23.8,24.0,24.0,23.8
2021-10-19,29.124060,-110.971318,2,22.7,22.6,22.3,22.9,22.8,22.5,22.8,22.7,22.6
2021-10-19,29.123985,-110.971286,2,22.7,22.6,22.3,22.9,22.8,22.5,22.8,22.7,22.6
2021-10-21,29.128777,-110.953926,3,26.7,26.6,26.3,26.6,26.5,26.3,26.5,26.4,26.3
2021-10-04,29.083809,-111.042101,4,30.7,30.5,30.3,30.7,30.6,30.4,30.6,30.5,30.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11,29.165260,-111.001808,3,20.2,20.1,19.8,20.3,20.2,20.0,20.2,20.2,20.1
2024-11-11,29.164831,-111.001746,3,20.2,20.1,19.8,20.3,20.2,20.0,20.2,20.2,20.1
2024-11-11,29.065486,-110.989319,7,20.2,20.1,19.8,20.3,20.2,20.0,20.2,20.2,20.1
